In [1]:
# Quick tutorial on using the CNN to get predictions for image tensors:

In [2]:
# Loading Packages: CNN
import torchvision
from torchvision import models
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer
import pytorch_lightning as pl
from torch.optim import Adam
from torch.utils.data import DataLoader
import pandas as pd
import yaml
import torch
from torchvision import transforms
import torch.utils.data
import pathlib
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import training_functions_classification as training_functions
import numpy as np
from tqdm import tqdm
import pytorch_lightning.metrics.functional as plm
from pytorch_lightning.loggers import NeptuneLogger, CometLogger
from torch.optim.lr_scheduler import MultiStepLR, CyclicLR, ReduceLROnPlateau
import typing
import argparse
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from facenet_pytorch import InceptionResnetV1
import matplotlib.pyplot as plt
import seaborn as sns

# Import model:
from run_arrest_multihead import MultiTaskClassification

# Setting up config:
global param_set_id, config
torch.manual_seed(912)
global param_set_id

# Importing Functions
import utils_convnet
# Loading Packages: GAN
import warnings
import argparse
import os
from PIL import Image
import numpy as np
import torch

# Additional Edits:
from PIL import ImageFont
from PIL import ImageDraw
import torch.nn as nn

In [3]:
# Path to model pkl: 
path_to_model = "/home/ec2-user/gan_cnn_latent_editing/latent_editing/combined_clean/gan_cnn_combined/model_checkpoints/multihead_0.0.3/checkpoint-model-epoch=38-val_auc_arrest_final_outcome=0.63.ckpt"
path_to_config = "/home/ec2-user/gan_cnn_latent_editing/latent_editing/combined_clean/gan_cnn_combined/configs/latent_editing-0.1.0.yaml"

In [4]:
config = yaml.load(
    open(path_to_config),
    Loader=yaml.FullLoader,
)

In [5]:
# Setting Device to CPU manually: 
gpu = []
device = torch.device(gpu[0] if gpu else "cpu")

In [7]:
#from run_arrest_multihead import MultiTaskClassification
def get_cnn(config, checkpoint_path):
    cnn = MultiTaskClassification(config)
    cnn.to(device)
    cnn.eval()
    # Weight transfer:
    if device.type == "cpu":
        checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    else:
        checkpoint = torch.load(checkpoint_path)
    cnn.load_state_dict(checkpoint["state_dict"])
    return cnn

In [8]:
# Loading in the CNN: 
cnn = get_cnn(config, path_to_model)

In [21]:
# Getting dummy predictions: 
test_img = torch.ones(1, 3, 512, 512)
pred_tensor = cnn.sigmoid(cnn(test_img)['arrest_final_outcome'])
pred_tensor.item()

0.9439283013343811

In [10]:
# Setting up the data-loader: 

In [11]:
'''Defining Data-Loader'''
image_directory = "/home/ec2-user/gan_cnn_latent_editing/latent_editing/latent-space-curvature-0.0.2/image_renders_seed_200"
images_full = [filename.stem for filename in pathlib.Path(image_directory).glob("*.png")]
images = images_full[1:10]
id_vector = np.arange(1, len(images)+1)
pred_df = pd.DataFrame(id_vector.reshape(-1, 1))
pred_df.columns = ["gen_image_id"]
pred_transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
pred_dataset = utils_convnet.MultiLabelPredictionImageDataset( 
    image_directory = image_directory,
    label_df=pred_df,
    id_var_name=config["id_var_name"],
    image_transformation=pred_transform)
pred_dataloader = torch.utils.data.DataLoader(pred_dataset, num_workers=32, batch_size=config["batch_size"])

In [12]:
with torch.no_grad():
    cnn.eval()
    for iteration, data in enumerate(pred_dataloader, 0):
        images, ids = data[0].to(device), data[1]
        for outcome in config["response_var_colnames"]:
            print("Predictions for: ", outcome)
            y_preds = cnn.sigmoid(cnn(images)[outcome])
            print(y_preds)

/home/ec2-user/gan_cnn_latent_editing/latent_editing/latent-space-curvature-0.0.2/image_renders_seed_200/seed_200_var_1.png
/home/ec2-user/gan_cnn_latent_editing/latent_editing/latent-space-curvature-0.0.2/image_renders_seed_200/seed_200_var_2.png
/home/ec2-user/gan_cnn_latent_editing/latent_editing/latent-space-curvature-0.0.2/image_renders_seed_200/seed_200_var_3.png
/home/ec2-user/gan_cnn_latent_editing/latent_editing/latent-space-curvature-0.0.2/image_renders_seed_200/seed_200_var_4.png
/home/ec2-user/gan_cnn_latent_editing/latent_editing/latent-space-curvature-0.0.2/image_renders_seed_200/seed_200_var_5.png
/home/ec2-user/gan_cnn_latent_editing/latent_editing/latent-space-curvature-0.0.2/image_renders_seed_200/seed_200_var_6.png
/home/ec2-user/gan_cnn_latent_editing/latent_editing/latent-space-curvature-0.0.2/image_renders_seed_200/seed_200_var_7.png
/home/ec2-user/gan_cnn_latent_editing/latent_editing/latent-space-curvature-0.0.2/image_renders_seed_200/seed_200_var_8.png
/home/ec